# Analisis de Sentimientos en reseñas de películas

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion1/7-sentiment-analysis.ipynb)

Ahora pongamos en práctica algunos de estos conceptos en un caso más real. Para esta práctica vamos a hacer un análisis de sentimientos sobre unas reseñas de películas. Este caso sería una simple clasificación binaria y podemos utilizar cualquier modelo para ese fin, lo adicional aquí es el pre-procesamiento de las entradas de texto.

### Referencias
* [Natural Language Processing in Action](https://www.manning.com/books/natural-language-processing-in-action)

In [1]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

/tmp/ipykernel_1660520/2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


Empecemos por cargar el dataset:

In [1]:
import pandas as pd
import numpy as np

reviews = pd.read_csv('./moviereviews.tsv', sep='\t')
reviews.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


Luego, hagamos algo de limpieza, vamos a remover nulos y valores vacíos:

In [2]:
reviews.dropna(inplace=True)
reviews.review = reviews.review.apply(lambda r: r.strip())
blanks = reviews[reviews.review == ''].index
reviews.drop(blanks, inplace=True)

In [3]:
reviews[reviews.review == ''].index

Index([], dtype='int64')

In [4]:
reviews.label.value_counts()

label
neg    969
pos    969
Name: count, dtype: int64

Tenemos un dataset balanceado de casi mil ejemplares por cada clase.

Para hacer las cosas simples, vamos a utilizar un VADER para computar el puntaje de positivo o negativo. Este modelo ya viene implementado dentro de NLTK.

In [5]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\joshu\AppData\Roaming\nltk_data...


True

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
reviews['scores'] = reviews.review.apply(lambda r: sid.polarity_scores(r))
reviews.head()

,label,review,scores
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co..."
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com..."
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com..."
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co..."
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co..."


Con estos puntajes ahora podemos convertir el resultado en una etiqueta de predicción:

In [7]:
reviews['compound'] = reviews.scores.apply(lambda s: s['compound'])    
reviews['prediction'] = reviews['compound'].apply(lambda c: 'pos' if c > 0 else 'neg')
reviews.head()

,label,review,scores,compound,prediction
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...",-0.9125,neg
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...",-0.8618,neg
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...",0.9951,pos
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...",0.9972,pos
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...",-0.2484,neg


Y finalmente computar unas cuantas métricas de calidad del modelo:

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_true = reviews.label.values
y_pred = reviews.prediction.values

acc = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred)


print(f"Accuracy:\n{acc}\n")
print(f"Classification Report:\n{cr}")
print(f"Confusion Matrix:\n{cm}")

Accuracy:
0.6357069143446853

Classification Report:
              precision    recall  f1-score   support

         neg       0.72      0.44      0.55       969
         pos       0.60      0.83      0.70       969

    accuracy                           0.64      1938
   macro avg       0.66      0.64      0.62      1938
weighted avg       0.66      0.64      0.62      1938

Confusion Matrix:
[[427 542]
 [164 805]]


La correctitud no es la mejor, aún podemos hacerlo mucho mejor que la línea base (50%). Parece que tenemos problemas con las etiquetas negativas!

La alternativa que encontramos fue probar con un modelo diferente a Vader, uno llamado Flair

In [9]:
#!pip install flair


from flair.models import TextClassifier
from flair.data import Sentence
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import time

c:\Users\joshu\Desktop\ICESI PLN\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Cargar datos
print("Cargando datos...")
reviews = pd.read_csv('./moviereviews.tsv', sep='\t')

reviews.dropna(inplace=True)
reviews.review = reviews.review.apply(lambda r: str(r).strip())
blanks = reviews[reviews.review == ''].index
reviews.drop(blanks, inplace=True)


print("\nAplicando VADER (línea base)...")
vader = SentimentIntensityAnalyzer()
reviews['vader_compound'] = reviews['review'].apply(lambda r: vader.polarity_scores(str(r))['compound'])
reviews['vader_prediction'] = reviews['vader_compound'].apply(lambda c: 'pos' if c > 0 else 'neg')

# Flair (mejora)
print("Cargando modelo Flair...")
classifier = TextClassifier.load('en-sentiment')
print("Modelo Flair cargado exitosamente")

print("Aplicando Flair...")

flair_sentiments = []
for i, review in enumerate(reviews['review']):
    if i % 500 == 0:
        print(f"Procesando reseña {i+1}/{len(reviews)}")
    
    sentence = Sentence(review)
    classifier.predict(sentence)
    
    # Extraer sentimiento
    if sentence.labels:
        label = sentence.labels[0]
        if label.value == 'POSITIVE':
            flair_sentiments.append(1.0)
        else:
            flair_sentiments.append(-1.0)
    else:
        flair_sentiments.append(0.0)


Cargando datos...

Aplicando VADER (línea base)...
Cargando modelo Flair...
Modelo Flair cargado exitosamente
Aplicando Flair...
Procesando reseña 1/1938
Procesando reseña 501/1938
Procesando reseña 1001/1938
Procesando reseña 1501/1938


In [11]:
reviews['flair_sentiment'] = flair_sentiments
reviews['flair_prediction'] = reviews['flair_sentiment'].apply(lambda c: 'pos' if c > 0 else 'neg')

# Evaluar VADER (línea base)
print("\n" + "="*60)
print("EVALUACIÓN VADER (LÍNEA BASE)")
print("="*60)

y_true = reviews['label'].values.astype(str)
y_pred_vader = reviews['vader_prediction'].values.astype(str)

vader_acc = accuracy_score(y_true, y_pred_vader)
vader_cm = confusion_matrix(y_true, y_pred_vader)
tn_v, fp_v, fn_v, tp_v = vader_cm.ravel()
vader_neg_recall = tn_v / (tn_v + fp_v) if (tn_v + fp_v) > 0 else 0
vader_pos_recall = tp_v / (tp_v + fn_v) if (tp_v + fn_v) > 0 else 0

print(f"VADER - Accuracy: {vader_acc:.3f}")
print(f"VADER - Neg Recall: {vader_neg_recall:.3f}")
print(f"VADER - Pos Recall: {vader_pos_recall:.3f}")

print(f"\nMatriz de Confusión VADER:")
print(vader_cm)

print(f"\nReporte de Clasificación VADER:")
print(classification_report(y_true, y_pred_vader))

# Evaluar Flair (mejora)
print("\n" + "="*60)
print("EVALUACIÓN FLAIR (MEJORA)")
print("="*60)

y_pred_flair = reviews['flair_prediction'].values.astype(str)

flair_acc = accuracy_score(y_true, y_pred_flair)
flair_cm = confusion_matrix(y_true, y_pred_flair)
tn_f, fp_f, fn_f, tp_f = flair_cm.ravel()
flair_neg_recall = tn_f / (tn_f + fp_f) if (tn_f + fp_f) > 0 else 0
flair_pos_recall = tp_f / (tp_f + fn_f) if (tp_f + fn_f) > 0 else 0

print(f"Flair - Accuracy: {flair_acc:.3f}")
print(f"Flair - Neg Recall: {flair_neg_recall:.3f}")
print(f"Flair - Pos Recall: {flair_pos_recall:.3f}")


EVALUACIÓN VADER (LÍNEA BASE)
VADER - Accuracy: 0.636
VADER - Neg Recall: 0.441
VADER - Pos Recall: 0.831

Matriz de Confusión VADER:
[[427 542]
 [164 805]]

Reporte de Clasificación VADER:
              precision    recall  f1-score   support

         neg       0.72      0.44      0.55       969
         pos       0.60      0.83      0.70       969

    accuracy                           0.64      1938
   macro avg       0.66      0.64      0.62      1938
weighted avg       0.66      0.64      0.62      1938


EVALUACIÓN FLAIR (MEJORA)
Flair - Accuracy: 0.805
Flair - Neg Recall: 0.966
Flair - Pos Recall: 0.645


In [13]:
print(f"\nReporte de Clasificación Flair:")
print(classification_report(y_true, y_pred_flair))

print("\n" + "="*60)
print("COMPARACIÓN FINAL: MEJORA SIGNIFICATIVA")
print("="*60)

print(f"MEJORA EN ACCURACY: {flair_acc - vader_acc:.3f} ({(flair_acc - vader_acc)*100:.1f}%)")
print(f"MEJORA EN NEG RECALL: {flair_neg_recall - vader_neg_recall:.3f} ({(flair_neg_recall - vader_neg_recall)*100:.1f}%)")
print(f"MEJORA EN POS RECALL: {flair_pos_recall - vader_pos_recall:.3f} ({(flair_pos_recall - vader_pos_recall)*100:.1f}%)")

print(f"\nRESUMEN:")
print(f"  VADER (línea base): Accuracy {vader_acc:.3f}, Neg Recall {vader_neg_recall:.3f}")
print(f"  Flair (mejora):     Accuracy {flair_acc:.3f}, Neg Recall {flair_neg_recall:.3f}")
print(f"  ¡MEJORA TOTAL:      +{(flair_acc - vader_acc)*100:.1f}% accuracy, +{(flair_neg_recall - vader_neg_recall)*100:.1f}% neg recall!")

print(f"\n" + "="*60)
print("EJEMPLOS DE MEJORA")
print("="*60)

corrected_cases = reviews[
    (reviews['vader_prediction'] != reviews['label']) & 
    (reviews['flair_prediction'] == reviews['label'])
].head(3)

if len(corrected_cases) > 0:
    print(f"\nCasos donde Flair corrigió VADER:")
    for i, (_, row) in enumerate(corrected_cases.iterrows()):
        print(f"\nEjemplo {i+1}:")
        print(f"Review: {row['review'][:120]}...")
        print(f"Label real: {row['label']}")
        print(f"VADER: {row['vader_prediction']} (compound: {row['vader_compound']:.3f})")
        print(f"Flair: {row['flair_prediction']} (sentiment: {row['flair_sentiment']:.3f})")


Reporte de Clasificación Flair:
              precision    recall  f1-score   support

         neg       0.73      0.97      0.83       969
         pos       0.95      0.64      0.77       969

    accuracy                           0.81      1938
   macro avg       0.84      0.81      0.80      1938
weighted avg       0.84      0.81      0.80      1938


COMPARACIÓN FINAL: MEJORA SIGNIFICATIVA
MEJORA EN ACCURACY: 0.170 (17.0%)
MEJORA EN NEG RECALL: 0.525 (52.5%)
MEJORA EN POS RECALL: -0.186 (-18.6%)

RESUMEN:
  VADER (línea base): Accuracy 0.636, Neg Recall 0.441
  Flair (mejora):     Accuracy 0.805, Neg Recall 0.966
  ¡MEJORA TOTAL:      +17.0% accuracy, +52.5% neg recall!

EJEMPLOS DE MEJORA

Casos donde Flair corrigió VADER:

Ejemplo 1:
Review: synopsis : melissa , a mentally-disturbed woman who likes to smoke , seduces doug , a minor-league baseball player who l...
Label real: neg
VADER: pos (compound: 0.987)
Flair: neg (sentiment: -1.000)

Ejemplo 2:
Review: tim robbins and ma

FLAIR HA MEJORADO SIGNIFICATIVAMENTE EL ANÁLISIS DE SENTIMIENTOS
Especialmente en la detección de reseñas negativas, que era nuestro objetivo principal